#EJERCICIO 3 
Apoyándose nuevamente en la sección de información. Recoge un texto extenso extraído, 
divídelos en fragmentos, luego realiza extracciones de sustantivos (POS) y categoriza estos 
sustantivos  (NER),  a  continuación  realiza  una  búsqueda  de  similitud  filtrando  por 
sustantivos, compara las distintas técnicas de distancias vistas en clases, elija la mejor y 
justifique la razón por la que esta técnica se ajusta para este tipo de búsquedas.

Para este ejercicio vamos a combinar 2 textos extraidos de playthroughs de canales en idioma inglés. Con esto se espera tener una fuente de texto lo suficientemente grande para que se generen clusters grandes y separados a partir del embedding para luego poder hacer la busqueda de similitud.

In [3]:
with open("./datos/informacion/JonGetsGames - Sagrada Full Playthrough  Video  BoardGameGeek.txt", 'r') as f:
    playthrough1 = f.read()

In [4]:
with open("./datos/informacion/Sagrada - Playthrough  Video  BoardGameGeek.txt", 'r') as f:
    playthrough2 = f.read()

In [5]:
#Removemos hasta donde empieza la transcripción
index1 = playthrough1.lower().find("transcripción:")
playthrough1 = playthrough1[index1 + len("transcripción:") + 1:]
#Ahora en el segundo video
index2 = playthrough2.lower().find("transcripción:")
playthrough2 = playthrough2[index2 + len("transcripción:") + 1:]

Ahora debemos separar los textos en fragmentos para luego poder realizar el embedding. Los textos al ser transcripciones de audio no tienen puntuación. Esto hace que no podamos obtener los "chunks" utilizando técnicas que involucren separar por puntuación.

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#Utilizamos el text splitter para dividir el texto en partes más pequeñas
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50) #Los valores son suficientemente grandes para que no se pierda información
texts = text_splitter.split_text(playthrough1)
texts2 = text_splitter.split_text(playthrough2)
#Unimos los dos textos
texts = texts + texts2

In [40]:
texts

["welcome to Jong Gates games today I'll be doing a full three-player playthrough of cigara in this game players will be constructing a colorful stained glass window in front of themselves by drafting color Dice from a central pool these dice must go down in front of them",
 "these dice must go down in front of them according to a variety of restrictions and they will also be trying to match several public objectives that everybody is shooting for I'll explain how the game actually plays while we're playing it so let's jump in here we have our",
 "it so let's jump in here we have our game fully set up for our three players we'll be playing the game for the perspective of the purple player over here I've already done several pieces of setup the most important being that each player has chosen a window pattern that",
 "player has chosen a window pattern that they've slid into their frame we each got two of these double-sided cards to choose from at the beginning of the game and then once

In [41]:
# Removemos todos los \n
for i in range(len(texts)):
    texts[i] = texts[i].replace("\n", " ")

In [12]:

import spacy
# Cargar el modelo de lenguaje preentrenado en español
nlp = spacy.load('en_core_web_lg')

In [43]:
#Vemos un ejemplo de cómo funciona
print(texts[0])
# Procesamos el primer texto
doc = nlp(texts[0])
# Imprimir las entidades nombradas, etiquetas y explicaciones
for ent in doc.ents:
    print(f'Entidad: {ent.text}, Etiqueta: {ent.label_}, Explicación: {spacy.explain(ent.label_)}')
for token in doc:
    print([token.text, token.pos_, spacy.explain(token.pos_)])                         

welcome to Jong Gates games today I'll be doing a full three-player playthrough of cigara in this game players will be constructing a colorful stained glass window in front of themselves by drafting color Dice from a central pool these dice must go down in front of them
Entidad: Jong Gates, Etiqueta: PERSON, Explicación: People, including fictional
Entidad: today, Etiqueta: DATE, Explicación: Absolute or relative dates or periods
Entidad: three, Etiqueta: CARDINAL, Explicación: Numerals that do not fall under another type
Entidad: Dice, Etiqueta: ORG, Explicación: Companies, agencies, institutions, etc.
['welcome', 'ADJ', 'adjective']
['to', 'ADP', 'adposition']
['Jong', 'PROPN', 'proper noun']
['Gates', 'PROPN', 'proper noun']
['games', 'NOUN', 'noun']
['today', 'NOUN', 'noun']
['I', 'PRON', 'pronoun']
["'ll", 'AUX', 'auxiliary']
['be', 'AUX', 'auxiliary']
['doing', 'VERB', 'verb']
['a', 'DET', 'determiner']
['full', 'ADJ', 'adjective']
['three', 'NUM', 'numeral']
['-', 'PUNCT', 'punc

In [50]:
import pandas as pd
# Creamos un DataFrame para almacenar los datos
df = pd.DataFrame(columns=['text_id', 'palabra', 'explicacion'])

In [51]:
for i,texto in enumerate(texts):
    # Procesar el texto con el modelo de spaCy
    doc = nlp(texto)
    # Iterar sobre los tokens en el Doc y agregar los detalles a la lista de datos
    for token in doc:
        # Verificar si el token es un sustantivo
        if token.pos_ != 'NOUN' and token.pos_ != 'PROPN':
            continue
        #Si el token es un sustantivo, lo agregamos al DataFrame
        df.loc[len(df)] = [i, token.text, spacy.explain(token.pos_)]
# Imprimir el DataFrame
print(df)

      text_id  palabra  explicacion
0           0     Jong  proper noun
1           0    Gates  proper noun
2           0    games         noun
3           0    today         noun
4           0   player         noun
...       ...      ...          ...
1409      192    Marty  proper noun
1410      192  sagrada  proper noun
1411      192     link         noun
1412      192   screen         noun
1413      192     game         noun

[1414 rows x 3 columns]


In [ ]:
import Levenshtein
# Creamos una funcion de busqueda para encontrar frases a partir de sustantivos
def buscar_frases_por_sustantivo(sustantivo, df=df, texts=texts):
    results = []
    #Utilizaremos distancia de levenshtein para encontrar coincidencias aproximadas
    for _, row in df.iterrows():
        # Calcular la distancia de Levenshtein entre el sustantivo y los sustantivos del DataFrame y lo almacenamos
        results.append((row['text_id'], row['palabra'], Levenshtein.distance(sustantivo, row['palabra'])))
    #Eliminamos repeticiones en los resultados
    results = list(set(results))
    # Ordenar los results por la distancia de Levenshtein
    results.sort(key=lambda x: x[2])
    # Mostramos las 5 oraciones más cercanas
    for i in range(5):
        if i < len(results):
            text_id, word, distance = results[i]
            print(f"Texto ID: {text_id}, Palabra: {word}, Distancia: {distance}")
            print(f"Frase: {texts[text_id]}")
            print()


In [88]:
# Ejemplo de uso
buscar_frases_por_sustantivo("pool")

Texto ID: 139, Palabra: pool, Distancia: 0
Frase: like after drafting you can return the dice of the bag and pull a new one out to choose the value on it and put it somewhere in your thing or put it in the draft pool if you're trying to deny somebody something after drafting re-roll the die if it still can't replace predict back in the draft pool

Texto ID: 0, Palabra: pool, Distancia: 0
Frase: welcome to Jong Gates games today I'll be doing a full three-player playthrough of cigara in this game players will be constructing a colorful stained glass window in front of themselves by drafting color Dice from a central pool these dice must go down in front of them

Texto ID: 183, Palabra: pool, Distancia: 0
Frase: it can't be I want anything but six five or three on this purple then it's a five so still can't take it it goes back to the pool which means I have to take this and just give up on the Robo and a stand there Marty cannot take either of those dice but as long as he pulls out a re